In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
rutor_url = 'http://5.45.84.133/new/'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[(my_df['up'] >= cut) | (my_df['down'] >= cut)]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
20,Зарубежные фильмы,13 Май 23,"Восстание зловещих мертвецов / Evil Dead Rise (2023) WEB-DL-HEVC 2160p | 4K | HDR | Dolby Vision Profile 8 | P, A",18.61 GB,57,42
24,Зарубежные фильмы,13 Май 23,Вакцина от коррупции (2022) WEB-DL 1080p,3.06 GB,94,24
29,Зарубежные фильмы,13 Май 23,Блуждающая Земля 2 / Liu lang di qiu 2 / The Wandering Earth II (2023) WEB-DLRip-AVC от DoMiNo & селезень | P | TVShows,2.34 GB,72,31
47,Зарубежные фильмы,13 Май 23,Свитуотер / Sweetwater (2023) WEB-DLRip от New-Team | Jaskier,1.46 GB,55,17
51,Зарубежные фильмы,13 Май 23,Свитуотер / Sweetwater (2023) WEB-DL 1080p от New-Team | Jaskier,8.89 GB,58,45
55,Зарубежные фильмы,13 Май 23,Золото Селины / Selina's Gold (2022) WEBRip | L2,750.17 MB,79,11
59,Зарубежные фильмы,13 Май 23,Air: Большой прыжок / Air (2023) WEBRip 1080p | Sub,6.04 GB,131,51
95,Зарубежные сериалы,13 Май 23,Чёрный рыцарь / Black Knight (Taekbaegisa) [S01] (2023) WEB-DL-HEVC 1080p от NewComers | Dolby Vision | HDR | P,8.60 GB,83,66
96,Зарубежные сериалы,13 Май 23,Чёрный рыцарь / Black Knight (Taekbaegisa) [S01] (2023) WEB-DLRip | NewComers,2.46 GB,76,49
97,Зарубежные сериалы,13 Май 23,Чёрный рыцарь / Black Knight (Taekbaegisa) [S01] (2023) WEB-DL 1080p | NewComers,10.79 GB,52,48


In [8]:
id = [226]
my_df.loc[id, 'href']

226    http://5.45.84.133/torrent/924351/resident-evil-4-deluxe-edition-build-11025382-dlcs-2023-pc-repack-ot-selezen
Name: href, dtype: object